In [1]:
!pip install tpot scikit-learn pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 10.5 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=74805b91dfc96861bcd92987b6e7fd990ad38662aab14c2868a0e32ecec0abcf
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tpot import TPOTClassifier
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dir_path = "/content/drive/My Drive/Machine Learning/obesity_risk_prediction/ObesityDataSet_raw_and_data_sinthetic.csv"

In [5]:
data = pd.read_csv(dir_path)

In [10]:
data.shape

(2111, 17)

In [14]:
data.columns

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [15]:
data.isnull().sum()

,0
Gender,0
Age,0
Height,0
Weight,0
family_history_with_overweight,0
FAVC,0
FCVC,0
NCP,0
CAEC,0
SMOKE,0


In [7]:
data_encoded = pd.get_dummies(data.drop('NObeyesdad', axis=1), drop_first=True)

In [8]:
data

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [9]:
data_encoded

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Male,family_history_with_overweight_yes,FAVC_yes,CAEC_Frequently,CAEC_Sometimes,CAEC_no,SMOKE_yes,SCC_yes,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,21.000000,1.620000,64.000000,2.0,3.0,2.000000,0.000000,1.000000,False,True,False,False,True,False,False,False,False,False,True,False,False,True,False
1,21.000000,1.520000,56.000000,3.0,3.0,3.000000,3.000000,0.000000,False,True,False,False,True,False,True,True,False,True,False,False,False,True,False
2,23.000000,1.800000,77.000000,2.0,3.0,2.000000,2.000000,1.000000,True,True,False,False,True,False,False,False,True,False,False,False,False,True,False
3,27.000000,1.800000,87.000000,3.0,3.0,2.000000,2.000000,0.000000,True,False,False,False,True,False,False,False,True,False,False,False,False,False,True
4,22.000000,1.780000,89.800000,2.0,1.0,2.000000,0.000000,0.000000,True,False,False,False,True,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,20.976842,1.710730,131.408528,3.0,3.0,1.728139,1.676269,0.906247,False,True,True,False,True,False,False,False,False,True,False,False,False,True,False
2107,21.982942,1.748584,133.742943,3.0,3.0,2.005130,1.341390,0.599270,False,True,True,False,True,False,False,False,False,True,False,False,False,True,False
2108,22.524036,1.752206,133.689352,3.0,3.0,2.054193,1.414209,0.646288,False,True,True,False,True,False,False,False,False,True,False,False,False,True,False
2109,24.361936,1.739450,133.346641,3.0,3.0,2.852339,1.139107,0.586035,False,True,True,False,True,False,False,False,False,True,False,False,False,True,False


In [20]:
# Hedef değişkeni (NObeyesdad) etiketleme
label_encoder = LabelEncoder()
data['NObeyesdad'] = label_encoder.fit_transform(data['NObeyesdad'])

In [21]:
data["NObeyesdad"]

,NObeyesdad
0,1
1,1
2,1
3,5
4,6
...,...
2106,4
2107,4
2108,4
2109,4


In [22]:
data["NObeyesdad"].value_counts()

,count
NObeyesdad,
2,351
4,324
3,297
5,290
6,290
1,287
0,272


In [23]:
# Özellikler (X) ve hedef değişkeni (y) ayırma
X = data_encoded
y = data['NObeyesdad']

In [24]:
# Eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# TPOT AutoML modelini tanımlama
tpot = TPOTClassifier(verbosity=2, generations=5, population_size=50, random_state=42)

Verdiğiniz kod, **TPOT** (Tree-based Pipeline Optimization Tool) adlı bir Python kütüphanesi kullanarak otomatik makine öğrenimi (AutoML) uygulaması için bir sınıflandırıcı oluşturmayı amaçlamaktadır. TPOT, verilen bir veri seti üzerinde en iyi makine öğrenimi modelini ve hiperparametrelerini bulmak için genetik algoritmalar kullanır. Kodda yer alan parametrelerin detaylarını inceleyelim:

### Kodun Detayları

```python
tpot = TPOTClassifier(verbosity=2, generations=5, population_size=50, random_state=42)
```

1. **TPOTClassifier**:
   - `TPOTClassifier`, sınıflandırma problemleri için kullanılan TPOT'ın bir alt sınıfıdır. Bu sınıf, farklı makine öğrenimi algoritmalarını ve özellik mühendisliğini birleştirerek en iyi modeli bulmayı amaçlar.

2. **verbosity=2**:
   - `verbosity`, TPOT'ın çıktılarının ne kadar ayrıntılı olacağını belirler.
     - `verbosity=0`: Hiçbir çıktı vermez.
     - `verbosity=1`: Süreç hakkında minimum bilgi verir.
     - `verbosity=2`: Daha ayrıntılı bilgiler sağlar (örneğin, her nesildeki en iyi modelin performansını gösterir).
   - Burada, ayrıntılı bilgiler almak için `verbosity=2` kullanılmıştır.

3. **generations=5**:
   - `generations`, TPOT'ın kaç nesil boyunca en iyi model kombinasyonlarını keşfedeceğini belirtir. Her nesil, mevcut en iyi modellerin üzerine inşa edilerek yeni modellerin oluşturulması anlamına gelir. Bu durumda, TPOT 5 nesil boyunca çalışacak ve her nesilde model performansını iyileştirmeye çalışacaktır.

4. **population_size=50**:
   - `population_size`, her nesilde kaç farklı model (birey) oluşturulacağını belirtir. Yani, her nesilde 50 farklı model kombinasyonu denenecektir. Daha büyük bir popülasyon, daha fazla çeşitlilik sağlayarak potansiyel olarak daha iyi sonuçlar elde edebilir.

5. **random_state=42**:
   - `random_state`, rastgele sayı üreteci için bir tohum değeri sağlar. Bu, sonuçların tekrarlanabilir olmasını sağlar. Yani, aynı veriler ve aynı ayarlar ile çalıştığınızda, modelin çıktısı her seferinde aynı olacaktır. `42`, genellikle örnek olarak kullanılan popüler bir değer olsa da, başka bir değer de kullanılabilir.

### Özet:
Bu kod, TPOT kullanarak sınıflandırma problemleri için otomatik model optimizasyonu yapmak üzere bir `TPOTClassifier` nesnesi oluşturur. Belirtilen ayarlarla, TPOT 5 nesil boyunca, her nesilde 50 farklı model deneyerek, en iyi performansı yakalamaya çalışacak ve işlem süreci hakkında ayrıntılı bilgiler verecektir.

In [26]:
tpot

TPOTClassifier(generations=5, population_size=50, random_state=42, verbosity=2)

In [27]:
tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9549786666198445

Generation 2 - Current best internal CV score: 0.9549786666198445

Generation 3 - Current best internal CV score: 0.9609028497181887

Generation 4 - Current best internal CV score: 0.968013976436711

Generation 5 - Current best internal CV score: 0.968013976436711

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=False, criterion=entropy, max_features=0.7000000000000001, min_samples_leaf=1, min_samples_split=3, n_estimators=100)


TPOTClassifier(generations=5, population_size=50, random_state=42, verbosity=2)

In [29]:
y_pred = tpot.predict(X_test)

In [30]:
print("Accuracy :", accuracy_score(y_test, y_pred))
print("\nClassification Report : \n ", classification_report(y_test, y_pred))

Accuracy : 0.9598108747044918

Classification Report : 
                precision    recall  f1-score   support

           0       0.93      0.96      0.95        56
           1       0.95      0.90      0.93        62
           2       0.96      0.97      0.97        78
           3       0.97      0.97      0.97        58
           4       1.00      1.00      1.00        63
           5       0.93      0.96      0.95        56
           6       0.98      0.94      0.96        50

    accuracy                           0.96       423
   macro avg       0.96      0.96      0.96       423
weighted avg       0.96      0.96      0.96       423

